In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
! gdown --id 1PUSJ56k93B42XW9oszcPDefM8HeJ2BOg
! pip -q uninstall -y kaggle
! pip -q install --upgrade pip
! pip -q install kaggle --upgrade
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets download aishikai/amc-2023

/usr/local/lib/python3.9/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1PUSJ56k93B42XW9oszcPDefM8HeJ2BOg
To: /content/kaggle.json
100% 64.0/64.0 [00:00<00:00, 108kB/s]
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 22.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.3/63.3 kB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
100% 860M/861M [00:42<00:00, 26.2MB/s]
100% 861M/861M [00:42<00:00, 21.3MB/s]


In [3]:
! unzip -q /content/amc-2023.zip -d data
! rm /content/amc-2023.zip

In [4]:
!pip install pandarallel

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 2.8 MB/s eta 0:00:00
  Created wheel for pandarallel: filename=pandarallel-1.6.4-py3-none-any.whl size=16677 sha256=60d18099723ecf6fe15976aed8252b2dc005fabe232a7654e19a145cad1a5523
  Stored in directory: /root/.cache/pip/wheels/41/01/29/deaa71fe596f8d857e57c4fb388db8861e23e6ed0b03204dcb
Successfully built pandarallel


In [5]:
import numpy as np
import polars as pl
import os
import pandas as pd
from tqdm import tqdm
import torch
import torch.nn.functional as F
from tqdm import tqdm
import gc
import multiprocessing
from time import time
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True)
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.preprocessing import StandardScaler

import random
import gc
tqdm.pandas()
pd.set_option('display.max_rows', 1000)


import warnings
warnings.filterwarnings("ignore")

INFO: Pandarallel will run on 4 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [6]:
train=pl.read_csv('/content/drive/MyDrive/AMC 2023/train.csv').to_pandas()
test=pl.read_csv('/content/drive/MyDrive/AMC 2023/test.csv').to_pandas()

train['TITLE']=train['TITLE'].fillna('')
test['TITLE']=test['TITLE'].fillna('')

train['DESCRIPTION']=train['DESCRIPTION'].fillna('')
test['DESCRIPTION']=test['DESCRIPTION'].fillna('')

train['BULLET_POINTS']=train['BULLET_POINTS'].fillna('')
test['BULLET_POINTS']=test['BULLET_POINTS'].fillna('')

test['PRODUCT_LENGTH']=np.nan

In [7]:
display(train.head())

,PRODUCT_ID,TITLE,BULLET_POINTS,DESCRIPTION,PRODUCT_TYPE_ID,PRODUCT_LENGTH,title_cluster_bert,brand_cluster_bert,bullet_cluster_bert
0,1925202,ArtzFolio Tulip Flowers Blackout Curtain for D...,[LUXURIOUS & APPEALING: Beautiful custom-made ...,,1650,2125.980000,1772,7782,4511
1,2673191,Marks & Spencer Girls' Pyjama Sets T86_2561C_N...,"[Harry Potter Hedwig Pyjamas (6-16 Yrs),100% c...",,2755,393.700000,1052,146,2189
2,2765088,PRIKNIK Horn Red Electric Air Horn Compressor ...,"[Loud Dual Tone Trumpet Horn, Compatible With ...","Specifications: Color: Red, Material: Aluminiu...",7537,748.031495,17,4706,4113
3,1594019,ALISHAH Women's Cotton Ankle Length Leggings C...,[Made By 95%cotton and 5% Lycra which gives yo...,AISHAH Women's Lycra Cotton Ankel Leggings. Br...,2996,787.401574,1933,736,7176
4,283658,The United Empire Loyalists: A Chronicle of th...,,,6112,598.424000,541,2049,3


In [8]:
import re
def extract_brand_words(text):
    words = re.findall(r'\b\w{1,}\b', text)
    return ' '.join(words[:1])

def extract_bullets_words(text):
    words = re.findall(r'\b\w{1,}\b', text)
    return ' '.join(words[:2])

def extract_desc_words(text):
    words = re.findall(r'\b\w{1,}\b', text)
    return ' '.join(words[:2])


train['BRAND']=train['TITLE'].parallel_apply(extract_brand_words)
test['BRAND']=test['TITLE'].parallel_apply(extract_brand_words)

train['BRAND_BULLET']=train['BULLET_POINTS'].parallel_apply(extract_bullets_words)
test['BRAND_BULLET']=test['BULLET_POINTS'].parallel_apply(extract_bullets_words)

train['BRAND_DESC']=train['DESCRIPTION'].parallel_apply(extract_desc_words)
test['BRAND_DESC']=test['DESCRIPTION'].parallel_apply(extract_desc_words)

train['text']=train['TITLE']+'|'+train['BULLET_POINTS']+'|'+train['DESCRIPTION']
test['text']=test['TITLE']+'|'+test['BULLET_POINTS']+'|'+test['DESCRIPTION']

In [9]:
train['bert_cluster']=train['brand_cluster_bert'].astype(str)+'|'+train['bullet_cluster_bert'].astype(str)+'|'+train['PRODUCT_TYPE_ID'].astype(str)
test['bert_cluster']=test['brand_cluster_bert'].astype(str)+'|'+test['bullet_cluster_bert'].astype(str)+'|'+test['PRODUCT_TYPE_ID'].astype(str)

In [10]:
train['BRAND_PRODUCT_TYPE']=train['BRAND']+'|'+train['PRODUCT_TYPE_ID'].astype(str)
test['BRAND_PRODUCT_TYPE']=test['BRAND']+'|'+test['PRODUCT_TYPE_ID'].astype(str)

In [11]:
train['BRAND_PRODUCT_TYPE']=train['BRAND']+'|'+train['PRODUCT_TYPE_ID'].astype(str)
test['BRAND_PRODUCT_TYPE']=test['BRAND']+'|'+test['PRODUCT_TYPE_ID'].astype(str)

In [12]:
train['BRAND_BULLET_PRODUCT_TYPE']=train['BRAND']+"|"+train['BRAND_BULLET']+"|"+train['PRODUCT_TYPE_ID'].astype(str)
test['BRAND_BULLET_PRODUCT_TYPE']=test['BRAND']+"|"+test['BRAND_BULLET']+"|"+test['PRODUCT_TYPE_ID'].astype(str)

In [13]:
train['BRAND_DESC_PRODUCT_TYPE']=train['BRAND']+"|"+train['BRAND_DESC']+"|"+train['PRODUCT_TYPE_ID'].astype(str)
test['BRAND_DESC_PRODUCT_TYPE']=test['BRAND']+"|"+test['BRAND_DESC']+"|"+test['PRODUCT_TYPE_ID'].astype(str)

In [14]:
train['DESC EQUAL BULLET']=(train['BRAND_DESC']==train['BRAND_BULLET'])
test['DESC EQUAL BULLET']=(test['BRAND_DESC']==test['BRAND_BULLET'])

In [15]:
from sklearn.preprocessing import LabelEncoder

train_df=pd.concat([train,test],axis=0)

le=LabelEncoder()
train_df['BRAND']=le.fit_transform(train_df['BRAND'])
train_df['BRAND_BULLET']=le.fit_transform(train_df['BRAND_BULLET'])
train_df['BRAND_DESC']=le.fit_transform(train_df['BRAND_DESC'])
train_df['BRAND_PRODUCT_TYPE']=le.fit_transform(train_df['BRAND_PRODUCT_TYPE'])
train_df['BRAND_BULLET_PRODUCT_TYPE']=le.fit_transform(train_df['BRAND_BULLET_PRODUCT_TYPE'])
train_df['BRAND_DESC_PRODUCT_TYPE']=le.fit_transform(train_df['BRAND_DESC_PRODUCT_TYPE'])
# train_df['text']=le.fit_transform(train_df['text'])

In [16]:
del train_df['TITLE'],train_df['BULLET_POINTS'],train_df['DESCRIPTION']
gc.collect()

19

In [17]:
train=train_df.iloc[:len(train)]
test=train_df.iloc[len(train):]

In [18]:
train['preds']=np.nan
test['preds']=np.nan

In [19]:
d_train=train.copy()

In [20]:
def split_list(numbers, num_splits):
    sublist_len = len(numbers) // num_splits

    # split the list into sublists
    sublists = [[] for i in range(num_splits)]
    for (i, num) in enumerate(numbers):
        sublists[i%num_splits].append(num)
    return sublists

In [21]:
a_train=pd.DataFrame()
a_test=pd.DataFrame()

def get_pred_value(value):
    idx, train, test = value
    for i_,i in enumerate(idx):
        if (i_%1000==0):
            print(f"Done:{i_}/{len(idx)}\n")
        temp=d_train[d_train['BRAND_BULLET_PRODUCT_TYPE']==i]['PRODUCT_LENGTH'].values
        ind=d_train[d_train['BRAND_BULLET_PRODUCT_TYPE']==i]['PRODUCT_LENGTH'].index

        val2=getVal(temp)
        mape,score2=comp_score(temp,[val2]*len(temp))
    #     preds[ind]=val2
        val=val2
        score=score2
        train.loc[train['BRAND_BULLET_PRODUCT_TYPE']==i,'preds']=min(val,10000)
        test.loc[test['BRAND_BULLET_PRODUCT_TYPE']==i,'preds']=min(val,10000)
    return (train,test)

def getVal(dist):
    median = np.power(np.mean(np.power(dist, -1)), -1)
    return median

def comp_score(y_true,y_pred):
    score=0
    score = mean_absolute_percentage_error(y_true,y_pred)                                      
    return score,max(0,100*(1-score))

label_enc=pd.DataFrame(train['BRAND_BULLET_PRODUCT_TYPE'].value_counts())
index=label_enc[label_enc['BRAND_BULLET_PRODUCT_TYPE']>2].index.values.tolist()
# create a pool of worker processes
print(len(index))
indexes = split_list(index,multiprocessing.cpu_count())
train_lists = [train.loc[train['BRAND_BULLET_PRODUCT_TYPE'].isin(index_)] for index_ in indexes]
test_lists = [test.loc[test['BRAND_BULLET_PRODUCT_TYPE'].isin(index_)] for index_ in indexes]
train_ = train.loc[~train['BRAND_BULLET_PRODUCT_TYPE'].isin(index)]
test_ = test.loc[~test['BRAND_BULLET_PRODUCT_TYPE'].isin(index)]
for i in range(len(indexes)):
    print(f"Shape: {train_lists[i].shape}")

for i in range(len(indexes)):
    print(f"Test Shape: {test_lists[i].shape}")
func_list = [(idx, train_, test_) for (idx, train_, test_) in zip(indexes, train_lists, test_lists)]

if __name__ == '__main__':
    with multiprocessing.Pool(multiprocessing.cpu_count()) as pool:
        result = pool.map(get_pred_value, func_list)
    train = pd.concat([result[i][0] for i in range(len(result))], axis = 0, ignore_index = True)
    test = pd.concat([result[i][1] for i in range(len(result))], axis = 0, ignore_index = True)
    train = pd.concat([train, train_], axis = 0, ignore_index = True)
    test = pd.concat([test, test_], axis = 0, ignore_index = True)
    print(train.shape)
    print(test.shape)
    a_train=train[train['preds'].notna()]    
    a_test=test[test['preds'].notna()]
    del train_, test_, train_lists, test_lists, indexes, func_list
    gc.collect()

115264
Shape: (129011, 16)
Shape: (128587, 16)
Shape: (127981, 16)
Shape: (126712, 16)
Shape: (124851, 16)
Shape: (124326, 16)
Shape: (123611, 16)
Shape: (123382, 16)
Test Shape: (37145, 16)
Test Shape: (36451, 16)
Test Shape: (36401, 16)
Test Shape: (36165, 16)
Test Shape: (35651, 16)
Test Shape: (35399, 16)
Test Shape: (35122, 16)
Test Shape: (34780, 16)
Done:0/14408

Done:0/14408

Done:0/14408

Done:0/14408

Done:0/14408

Done:0/14408

Done:0/14408

Done:0/14408

Done:1000/14408

Done:1000/14408

Done:1000/14408

Done:1000/14408

Done:1000/14408

Done:1000/14408

Done:1000/14408

Done:1000/14408

Done:2000/14408

Done:2000/14408

Done:2000/14408

Done:2000/14408

Done:2000/14408

Done:2000/14408

Done:2000/14408

Done:3000/14408

Done:2000/14408

Done:3000/14408

Done:3000/14408

Done:3000/14408

Done:3000/14408

Done:4000/14408

Done:3000/14408

Done:3000/14408

Done:3000/14408

Done:4000/14408

Done:4000/14408

Done:4000/14408

Done:4000/14408

Done:5000/14408

Done:4000/14408

Do

In [22]:
print(comp_score(train[train['preds'].notna()]['PRODUCT_LENGTH'].values,train[train['preds'].notna()]['preds']))
print(comp_score(a_train[a_train['preds'].notna()]['PRODUCT_LENGTH'].values,a_train[a_train['preds'].notna()]['preds']))
print(len(a_train),len(train))

(0.2188076169578528, 78.11923830421472)
(0.2188076169578528, 78.11923830421472)
1008461 2249686


In [23]:
train=train[train['preds'].isna()]
test=test[test['preds'].isna()]

def get_pred_value(value):
    idx, train, test = value
    for i_,i in enumerate(idx):
        if (i_%1000==0):
            print(f"Done:{i_}/{len(idx)}\n")
        temp=d_train[d_train['BRAND_DESC_PRODUCT_TYPE']==i]['PRODUCT_LENGTH'].values
        ind=d_train[d_train['BRAND_DESC_PRODUCT_TYPE']==i]['PRODUCT_LENGTH'].index

        val2=getVal(temp)
        mape,score2=comp_score(temp,[val2]*len(temp))
    #     preds[ind]=val2
        val=val2
        score=score2
        train.loc[train['BRAND_DESC_PRODUCT_TYPE']==i,'preds']=min(val,10000)
        test.loc[test['BRAND_DESC_PRODUCT_TYPE']==i,'preds']=min(val,10000)
    return (train,test)

def getVal(dist):
    median = np.power(np.mean(np.power(dist, -1)), -1)
    return median

def comp_score(y_true,y_pred):
    score=0
    score = mean_absolute_percentage_error(y_true,y_pred)                                      
    return score,max(0,100*(1-score))

label_enc=pd.DataFrame(train['BRAND_DESC_PRODUCT_TYPE'].value_counts())
index=label_enc[label_enc['BRAND_DESC_PRODUCT_TYPE']>2].index.values.tolist()
# create a pool of worker processes
print(len(index))
indexes = split_list(index,multiprocessing.cpu_count())
train_lists = [train.loc[train['BRAND_DESC_PRODUCT_TYPE'].isin(index_)] for index_ in indexes]
test_lists = [test.loc[test['BRAND_DESC_PRODUCT_TYPE'].isin(index_)] for index_ in indexes]
train_ = train.loc[~train['BRAND_DESC_PRODUCT_TYPE'].isin(index)]
test_ = test.loc[~test['BRAND_DESC_PRODUCT_TYPE'].isin(index)]
for i in range(len(indexes)):
    print(f"Shape: {train_lists[i].shape}")

for i in range(len(indexes)):
    print(f"Test Shape: {test_lists[i].shape}")
func_list = [(idx, train_, test_) for (idx, train_, test_) in zip(indexes, train_lists, test_lists)]

if __name__ == '__main__':
    with multiprocessing.Pool(multiprocessing.cpu_count()) as pool:
        result = pool.map(get_pred_value, func_list)
    train = pd.concat([result[i][0] for i in range(len(result))], axis = 0, ignore_index = True)
    test = pd.concat([result[i][1] for i in range(len(result))], axis = 0, ignore_index = True)
    train = pd.concat([train, train_], axis = 0, ignore_index = True)
    test = pd.concat([test, test_], axis = 0, ignore_index = True)
    print(train.shape)
    print(test.shape)
    a_train=pd.concat([a_train,train[train['preds'].notna()]],axis=0)    
    a_test=pd.concat([a_test,test[test['preds'].notna()]],axis=0) 
    del train_, test_, train_lists, test_lists, indexes, func_list
    gc.collect()

25894
Shape: (16541, 16)
Shape: (16253, 16)
Shape: (16202, 16)
Shape: (16172, 16)
Shape: (16119, 16)
Shape: (16095, 16)
Shape: (16073, 16)
Shape: (16062, 16)
Test Shape: (4993, 16)
Test Shape: (4771, 16)
Test Shape: (4844, 16)
Test Shape: (4845, 16)
Test Shape: (4784, 16)
Test Shape: (4909, 16)
Test Shape: (4724, 16)
Test Shape: (4626, 16)
Done:0/3237

Done:0/3237

Done:0/3237

Done:0/3237

Done:0/3237

Done:0/3237

Done:0/3236

Done:0/3236

Done:1000/3237

Done:1000/3237

Done:1000/3237

Done:1000/3237

Done:1000/3237

Done:1000/3237

Done:1000/3236

Done:1000/3236

Done:2000/3237

Done:2000/3237

Done:2000/3237

Done:2000/3237

Done:2000/3237

Done:2000/3237

Done:2000/3236

Done:2000/3236

Done:3000/3237

Done:3000/3237

Done:3000/3237

Done:3000/3237

Done:3000/3237

Done:3000/3237

Done:3000/3236

Done:3000/3236

(1241225, 16)
(447622, 16)


In [24]:
comp_score(train[train['preds'].notna()]['PRODUCT_LENGTH'].values,train[train['preds'].notna()]['preds'])

(0.33458760296883067, 66.54123970311694)

In [25]:
train=train[train['preds'].isna()]
test=test[test['preds'].isna()]

def get_pred_value(value):
    idx, train, test = value
    for i_,i in enumerate(idx):
        if (i_%1000==0):
            print(f"Done:{i_}/{len(idx)}\n")
        temp=d_train[d_train['BRAND_PRODUCT_TYPE']==i]['PRODUCT_LENGTH'].values
        ind=d_train[d_train['BRAND_PRODUCT_TYPE']==i]['PRODUCT_LENGTH'].index

        val2=getVal(temp)
        mape,score2=comp_score(temp,[val2]*len(temp))
    #     preds[ind]=val2
        val=val2
        score=score2
        train.loc[train['BRAND_PRODUCT_TYPE']==i,'preds']=min(val,10000)
        test.loc[test['BRAND_PRODUCT_TYPE']==i,'preds']=min(val,10000)
    return (train,test)

def getVal(dist):
    median = np.power(np.mean(np.power(dist, -1)), -1)
    return median

def comp_score(y_true,y_pred):
    score=0
    score = mean_absolute_percentage_error(y_true,y_pred)                                      
    return score,max(0,100*(1-score))

label_enc=pd.DataFrame(train['BRAND_PRODUCT_TYPE'].value_counts())
index=label_enc[label_enc['BRAND_PRODUCT_TYPE']>1].index.values.tolist()
# create a pool of worker processes
print(len(index))
indexes = split_list(index,multiprocessing.cpu_count())
train_lists = [train.loc[train['BRAND_PRODUCT_TYPE'].isin(index_)] for index_ in indexes]
test_lists = [test.loc[test['BRAND_PRODUCT_TYPE'].isin(index_)] for index_ in indexes]
train_ = train.loc[~train['BRAND_PRODUCT_TYPE'].isin(index)]
test_ = test.loc[~test['BRAND_PRODUCT_TYPE'].isin(index)]
for i in range(len(indexes)):
    print(f"Shape: {train_lists[i].shape}")

for i in range(len(indexes)):
    print(f"Test Shape: {test_lists[i].shape}")
func_list = [(idx, train_, test_) for (idx, train_, test_) in zip(indexes, train_lists, test_lists)]

if __name__ == '__main__':
    with multiprocessing.Pool(multiprocessing.cpu_count()) as pool:
        result = pool.map(get_pred_value, func_list)
    train = pd.concat([result[i][0] for i in range(len(result))], axis = 0, ignore_index = True)
    test = pd.concat([result[i][1] for i in range(len(result))], axis = 0, ignore_index = True)
    train = pd.concat([train, train_], axis = 0, ignore_index = True)
    test = pd.concat([test, test_], axis = 0, ignore_index = True)
    print(train.shape)
    print(test.shape)
    a_train=pd.concat([a_train,train[train['preds'].notna()]],axis=0)    
    a_test=pd.concat([a_test,test[test['preds'].notna()]],axis=0) 
    del train_, test_, train_lists, test_lists, indexes, func_list
    gc.collect()

159284
Shape: (53304, 16)
Shape: (53281, 16)
Shape: (53262, 16)
Shape: (53239, 16)
Shape: (53229, 16)
Shape: (53200, 16)
Shape: (53181, 16)
Shape: (53164, 16)
Test Shape: (13333, 16)
Test Shape: (13448, 16)
Test Shape: (13334, 16)
Test Shape: (13050, 16)
Test Shape: (13120, 16)
Test Shape: (13167, 16)
Test Shape: (13131, 16)
Test Shape: (13099, 16)
Done:0/19911

Done:0/19911

Done:0/19911

Done:0/19911

Done:0/19910

Done:0/19910

Done:0/19910

Done:0/19910

Done:1000/19911

Done:1000/19911

Done:1000/19911

Done:1000/19911

Done:1000/19910

Done:1000/19910

Done:1000/19910

Done:1000/19910

Done:2000/19911

Done:2000/19911

Done:2000/19911

Done:2000/19911

Done:2000/19910

Done:2000/19910

Done:2000/19910

Done:2000/19910

Done:3000/19911

Done:3000/19911

Done:3000/19911

Done:3000/19911

Done:3000/19910

Done:3000/19910

Done:3000/19910

Done:3000/19910

Done:4000/19911

Done:4000/19911

Done:4000/19911

Done:4000/19911

Done:4000/19910

Done:4000/19910

Done:4000/19910

Done:4000/

In [26]:
train=train[train['preds'].isna()]
test=test[test['preds'].isna()]

def get_pred_value(value):
    idx, train, test = value
    for i_,i in enumerate(idx):
        if (i_%1000==0):
            print(f"Done:{i_}/{len(idx)}\n")
        temp=d_train[d_train['bert_cluster']==i]['PRODUCT_LENGTH'].values
        ind=d_train[d_train['bert_cluster']==i]['PRODUCT_LENGTH'].index

        val2=getVal(temp)
        mape,score2=comp_score(temp,[val2]*len(temp))
    #     preds[ind]=val2
        val=val2
        score=score2
        train.loc[train['bert_cluster']==i,'preds']=min(val,10000)
        test.loc[test['bert_cluster']==i,'preds']=min(val,10000)
    return (train,test)

def getVal(dist):
    median = np.power(np.mean(np.power(dist, -1)), -1)
    return median

def comp_score(y_true,y_pred):
    score=0
    score = mean_absolute_percentage_error(y_true,y_pred)                                      
    return score,max(0,100*(1-score))

label_enc=pd.DataFrame(train['bert_cluster'].value_counts())
index=label_enc[label_enc['bert_cluster']>2].index.values.tolist()
# create a pool of worker processes
print(len(index))
indexes = split_list(index,multiprocessing.cpu_count())
train_lists = [train.loc[train['bert_cluster'].isin(index_)] for index_ in indexes]
test_lists = [test.loc[test['bert_cluster'].isin(index_)] for index_ in indexes]
train_ = train.loc[~train['bert_cluster'].isin(index)]
test_ = test.loc[~test['bert_cluster'].isin(index)]
for i in range(len(indexes)):
    print(f"Shape: {train_lists[i].shape}")

for i in range(len(indexes)):
    print(f"Test Shape: {test_lists[i].shape}")
func_list = [(idx, train_, test_) for (idx, train_, test_) in zip(indexes, train_lists, test_lists)]

if __name__ == '__main__':
    with multiprocessing.Pool(multiprocessing.cpu_count()) as pool:
        result = pool.map(get_pred_value, func_list)
    train = pd.concat([result[i][0] for i in range(len(result))], axis = 0, ignore_index = True)
    test = pd.concat([result[i][1] for i in range(len(result))], axis = 0, ignore_index = True)
    train = pd.concat([train, train_], axis = 0, ignore_index = True)
    test = pd.concat([test, test_], axis = 0, ignore_index = True)
    print(train.shape)
    print(test.shape)
    a_train=pd.concat([a_train,train[train['preds'].notna()]],axis=0)    
    a_test=pd.concat([a_test,test[test['preds'].notna()]],axis=0) 
    del train_, test_, train_lists, test_lists, indexes, func_list
    gc.collect()

26464
Shape: (26515, 16)
Shape: (26319, 16)
Shape: (26208, 16)
Shape: (26166, 16)
Shape: (26076, 16)
Shape: (25972, 16)
Shape: (25925, 16)
Shape: (25881, 16)
Test Shape: (10242, 16)
Test Shape: (10292, 16)
Test Shape: (9982, 16)
Test Shape: (10230, 16)
Test Shape: (9869, 16)
Test Shape: (9872, 16)
Test Shape: (9967, 16)
Test Shape: (9962, 16)
Done:0/3308

Done:0/3308

Done:0/3308

Done:0/3308

Done:0/3308

Done:0/3308

Done:0/3308

Done:0/3308

Done:1000/3308

Done:1000/3308

Done:1000/3308

Done:1000/3308

Done:1000/3308

Done:1000/3308

Done:1000/3308

Done:1000/3308

Done:2000/3308

Done:2000/3308

Done:2000/3308

Done:2000/3308

Done:2000/3308

Done:2000/3308

Done:2000/3308

Done:2000/3308

Done:3000/3308

Done:3000/3308

Done:3000/3308

Done:3000/3308

Done:3000/3308

Done:3000/3308

Done:3000/3308

Done:3000/3308

(685848, 16)
(303444, 16)


In [27]:
comp_score(train[train['preds'].notna()]['PRODUCT_LENGTH'].values,train[train['preds'].notna()]['preds'])

(0.35955215092966547, 64.04478490703345)

In [28]:
comp_score(a_train[a_train['preds'].notna()]['PRODUCT_LENGTH'].values,a_train[a_train['preds'].notna()]['preds'])

(0.26486020985777764, 73.51397901422223)

In [29]:
len(d_train)

2249686

In [30]:
len(a_train)

1772900

In [31]:
train=train[train['preds'].isna()]
test=test[test['preds'].isna()]
def getVal(dist):
    median = np.power(np.mean(np.power(dist, -1)), -1)
    return median

def comp_score(y_true,y_pred):
    score=0
    score = mean_absolute_percentage_error(y_true,y_pred)                                      
    return score,max(0,100*(1-score))

temp=d_train.groupby(['BRAND'])['PRODUCT_LENGTH'].agg(['count','min','max'])
temp['diff']=(temp['max']-temp['min'])/temp['min']
brands_smaller_1=temp[(temp['diff']<=1.7)&(temp['count']>7)].index

for i in tqdm(set(brands_smaller_1.tolist()+[1093387,568826,368938])):
    temp=d_train[d_train['BRAND']==i]['PRODUCT_LENGTH'].values
    ind=d_train[d_train['BRAND']==i]['PRODUCT_LENGTH'].index
     
    if(len(ind)==0):
        continue
            
    val2=getVal(temp)
    mape,score2=comp_score(temp,[val2]*len(temp))
    val=val2
    score=score2
        
    if(i==1093387 | i== 568826):
        val=590.55118
        
    if(i==368938):
        val=393.700787
        
    train.loc[(train['BRAND']==i),'preds']=min(val, 10000)
    test.loc[(test['BRAND']==i),'preds']=min(val, 10000)
    
a_train=pd.concat([a_train,train[train['preds'].notna()]],axis=0)    
a_test=pd.concat([a_test,test[test['preds'].notna()]],axis=0)

100%|██████████| 9104/9104 [01:10<00:00, 128.48it/s]


In [32]:
# train=train[train['preds'].isna()]
# test=test[test['preds'].isna()]

# def getVal(dist):
#     median = np.power(np.mean(np.power(dist, -1)), -1)
#     return median



# categories= ['watch','clock','print','poster','mixture','soap','supplement','food','protein','vitamin','baby','kid','child','queen','tv','king','copper', 'diamond', 'gold', 'aluminum', 'wood', 'stick', 'bat', 'ball', 'mobile', 'phone', 'cloth', 'book', 'novel', 'guide', 'letter', 'wear', 'men', 'women', 'beauty', 'paper', 'small', 'big', 'xl', 'medium', 'large', 'health', 'suit', 'spring', 'summer', 'winter', 'electronics', 'household', 'food', 'beverage', 'baby', 'pet', 'outdoor', 'improvement', 'tool', 'furniture', 'jewelry', 'watch', 'office', 'supply', 'craft', 'art', 'music', 'vehicle', 'accessory', 'game', 'party', 'decor', 'travel', 'luggage', 'camera', 'photograph', 'security', 'automatic', 'garden', 'plant', 'flower']

# total=0
# for cat in tqdm(categories):
#     temp=train[(train['text'].str.contains(cat))].groupby(['PRODUCT_TYPE_ID'])['PRODUCT_LENGTH'].agg(['min','max','count',getVal])
#     temp['diff']=temp['max']-temp['min']
#     temp=temp[(temp['count']>2)&(temp['diff']<2*temp['min'])]
    
#     values=temp['getVal'].to_dict()
#     train.loc[(train['text'].str.contains(cat))&(train['PRODUCT_TYPE_ID'].isin(values.keys())),'preds']=train.loc[(train['text'].str.contains(cat))&(train['PRODUCT_TYPE_ID'].isin(values.keys())),'PRODUCT_TYPE_ID'].apply(lambda x:values[x])
#     test.loc[(test['text'].str.contains(cat))&(test['PRODUCT_TYPE_ID'].isin(values.keys())),'preds']=test.loc[(test['text'].str.contains(cat))&(test['PRODUCT_TYPE_ID'].isin(values.keys())),'PRODUCT_TYPE_ID'].apply(lambda x:values[x])
#     print(train.shape)
#     print(test.shape)
# a_train=pd.concat([a_train,train[train['preds'].notna()]],axis=0)    
# a_test=pd.concat([a_test,test[test['preds'].notna()]],axis=0) 

In [33]:
train=train[train['preds'].isna()]
test=test[test['preds'].isna()]
def get_pred_value(value):
    idx, train, test = value
    for i_,i in enumerate(idx):
        if (i_%1000==0):
            print(f"Done:{i_}/{len(idx)}\n")
        temp=d_train[d_train['PRODUCT_TYPE_ID']==i]['PRODUCT_LENGTH'].values
        ind=d_train[d_train['PRODUCT_TYPE_ID']==i]['PRODUCT_LENGTH'].index

        val2=getVal(temp)
        mape,score2=comp_score(temp,[val2]*len(temp))
    #     preds[ind]=val2
        val=val2
        score=score2
        train.loc[train['PRODUCT_TYPE_ID']==i,'preds']=min(val,10000)
        test.loc[test['PRODUCT_TYPE_ID']==i,'preds']=min(val,10000)
    return (train,test)

def getVal(dist):
    median = np.power(np.mean(np.power(dist, -1)), -1)
    return median

def comp_score(y_true,y_pred):
    score=0
    score = mean_absolute_percentage_error(y_true,y_pred)                                      
    return score,max(0,100*(1-score))

label_enc=pd.DataFrame(train['PRODUCT_TYPE_ID'].value_counts())
index=label_enc[label_enc['PRODUCT_TYPE_ID']>0].index.values.tolist()
# create a pool of worker processes
print(len(index))
indexes = split_list(index,multiprocessing.cpu_count())
train_lists = [train.loc[train['PRODUCT_TYPE_ID'].isin(index_)] for index_ in indexes]
test_lists = [test.loc[test['PRODUCT_TYPE_ID'].isin(index_)] for index_ in indexes]
train_ = train.loc[~train['PRODUCT_TYPE_ID'].isin(index)]
test_ = test.loc[~test['PRODUCT_TYPE_ID'].isin(index)]
for i in range(len(indexes)):
    print(f"Shape: {train_lists[i].shape}")

for i in range(len(indexes)):
    print(f"Test Shape: {test_lists[i].shape}")
func_list = [(idx, train_, test_) for (idx, train_, test_) in zip(indexes, train_lists, test_lists)]

if __name__ == '__main__':
    with multiprocessing.Pool(multiprocessing.cpu_count()) as pool:
        result = pool.map(get_pred_value, func_list)
    train = pd.concat([result[i][0] for i in range(len(result))], axis = 0, ignore_index = True)
    test = pd.concat([result[i][1] for i in range(len(result))], axis = 0, ignore_index = True)
    train = pd.concat([train, train_], axis = 0, ignore_index = True)
    test = pd.concat([test, test_], axis = 0, ignore_index = True)
    print(train.shape)
    print(test.shape)
    a_train=pd.concat([a_train,train[train['preds'].notna()]],axis=0)    
    a_test=pd.concat([a_test,test[test['preds'].notna()]],axis=0) 
    del train_, test_, train_lists, test_lists, indexes, func_list
    gc.collect()

12559
Shape: (61045, 16)
Shape: (57637, 16)
Shape: (56491, 16)
Shape: (56110, 16)
Shape: (55645, 16)
Shape: (55417, 16)
Shape: (55186, 16)
Shape: (55015, 16)
Test Shape: (28711, 16)
Test Shape: (26390, 16)
Test Shape: (26548, 16)
Test Shape: (25521, 16)
Test Shape: (25549, 16)
Test Shape: (25484, 16)
Test Shape: (25249, 16)
Test Shape: (25739, 16)
Done:0/1570

Done:0/1570

Done:0/1570

Done:0/1570

Done:0/1570

Done:0/1570

Done:0/1570

Done:0/1569

Done:1000/1570

Done:1000/1570

Done:1000/1570

Done:1000/1570

Done:1000/1570

Done:1000/1570

Done:1000/1570

Done:1000/1569

(452546, 16)
(209881, 16)


In [34]:
comp_score(train[train['preds'].notna()]['PRODUCT_LENGTH'].values,train[train['preds'].notna()]['preds'])

(0.9481973954341101, 5.180260456588992)

In [35]:
comp_score(a_train[a_train['preds'].notna()]['PRODUCT_LENGTH'].values,a_train[a_train['preds'].notna()]['preds'])

(0.40097227420345244, 59.90277257965475)

In [36]:
train=train[train['preds'].isna()]
test=test[test['preds'].isna()]


hm=getVal(d_train['PRODUCT_LENGTH'])

train['preds'].fillna(hm,inplace=True)
test['preds'].fillna(hm,inplace=True)


a_train=pd.concat([a_train,train[train['preds'].notna()]],axis=0)    
a_test=pd.concat([a_test,test[test['preds'].notna()]],axis=0)    


In [37]:
len(a_test)

734736

In [38]:
a_test['PRODUCT_LENGTH']=a_test['preds']
a_test = a_test.drop_duplicates(subset= ["PRODUCT_ID"], keep='last')
submission=pd.read_csv('/content/data/dataset/sample_submission.csv')
submission.rename(columns={'PRODUCT_LENGTH':'L'},inplace=True)
submission=submission.merge(a_test[['PRODUCT_ID','PRODUCT_LENGTH']],on='PRODUCT_ID')
submission[['PRODUCT_ID','PRODUCT_LENGTH']].to_csv('submission.csv',index=False)

In [39]:
submission

,PRODUCT_ID,L,PRODUCT_LENGTH
0,604373,701.093794,709.852057
1,1729783,734.506163,881.189143
2,1871949,741.360258,1000.000000
3,1107571,730.327767,90.946316
4,624253,666.847946,506.967964
...,...,...,...
734731,921419,733.838809,1060.263493
734732,2456362,746.810825,612.000000
734733,841529,691.127128,116.927131
734734,1190194,757.643591,885.106383
